In [ ]:
# Install libraries
!pip install datasets
!pip install transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# wandb - Create account on and log in to https://wandb.ai/, then paste the access token
import wandb
wandb.login()

True

In [ ]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
import torch
from torch.utils.data import random_split
from transformers import GPT2Tokenizer

In [ ]:
# Load moral_stories dataset from Hugging Face - https://huggingface.co/datasets/demelin/moral_stories
dataset=load_dataset('demelin/moral_stories','full')

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
X_train = dataset['train']['norm'][:8000]
X_test = dataset['train']['norm'][-2000:]

y_train = dataset['train']['label'][:8000]
y_test = dataset['train']['label'][-2000:]
print("Total Dataset (Including Validation) - ", len(dataset['train']))
print("Train Dataset - ",len(X_train),'\n',"Test Dataset - ",len(X_test))

Total Dataset (Including Validation) -  12000
Train Dataset -  8000 
 Test Dataset -  2000


In [ ]:
%env WANDB_PROJECT=hw7_2

env: WANDB_PROJECT=hw7_2


# Create GPT-2 tokenizer and define model

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12

# Tokenize data and split dataset into training and validation

In [ ]:
max_length = max([len(tokenizer.encode(x)) for x in X_train])

In [ ]:
#added to solve error
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
class moral():
    def __init__(self, x, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        for txt in x:
            encodings_dict = tokenizer(txt, max_length= max_length, pad_to_max_length=True, return_tensors='pt')
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


In [ ]:
# Split data 80-20 to form training and validation data
dataset = moral(X_train, tokenizer, max_length=max_length)
train_size = int(0.8 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

<ipython-input-83-3e7ea2d15a0f>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
<ipython-input-83-3e7ea2d15a0f>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))


# Define training parameters

In [ ]:
%env WANDB_WATCH=all
%env WANDB_SILENT=true

env: WANDB_WATCH=all
env: WANDB_SILENT=true


In [ ]:
# Refer to wandb documentation to log training https://docs.wandb.ai/guides/integrations/huggingface
training_args = TrainingArguments(output_dir = '/outputs_4',
    learning_rate = 0.0001,
    num_train_epochs=4,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,   
    weight_decay=0.002,              
    report_to="wandb",
    run_name="training_4")

PyTorch: setting up devices


# Train model

In [ ]:
'''<<Insert code for trainer here>>'''
model = GPT2LMHeadModel.from_pretrained('gpt2')
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, 
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/f27b190eeac4c2302d24068eabf5e9d6044389ae/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.25.1",
  "use_cach

Epoch,Training Loss,Validation Loss
1,No log,1.428599
2,1.518400,1.420891
3,1.200500,1.457809
4,1.054200,1.498110


***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
Saving model checkpoint to /outputs_4/checkpoint-500
Configuration saved in /outputs_4/checkpoint-500/config.json
Model weights saved in /outputs_4/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
Saving model checkpoint to /outputs_4/checkpoint-1000
Configuration saved in /outputs_4/checkpoint-1000/config.json
Model weights saved in /outputs_4/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16
Saving model checkpoint to /outputs_4/checkpoint-1500
Configuration saved in /outputs_4/checkpoint-1500/config.json
Model weights saved in /outputs_4/checkpoint-1500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1600
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
wandb.finish()

eval/loss,▁▁▃▅█
eval/runtime,▇▇█▁▂
eval/samples_per_second,▂▂▁█▇
eval/steps_per_second,▂▂▁█▇
train/epoch,▁▁▃▄▅▆▆███
train/global_step,▁▁▃▄▅▆▆███
train/learning_rate,█▆▃▁
train/loss,█▄▂▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


# Inference using sample data

In [ ]:
'''<<Insert code for Tokenize sample data and use model.generate function to get predicted logits and decode them using tokenizer.decode function refer hugging face documentation for further information >>'''
input = "My Parents are"
encoded_input = tokenizer(input, return_tensors='pt').to('cuda:0')
outputs = model.generate(encoded_input.input_ids, attention_mask = encoded_input.attention_mask, do_sample=False, max_length=30, temperature = 1, top_k = 20, top_p = 1).to('cuda:0')
tokenizer.batch_decode(outputs, skip_special_tokens=True)


# Inference using test dataset

In [ ]:
'''<<Insert code for infering every sentence in test dataset X_test >>'''
from tqdm import tqdm

results = []
for i in tqdm(range(len(X_test))):
  sample = X_test[i].split()
  sample = " ".join(sample[:len(sample)//2])
  encoded_input = tokenizer(sample, return_tensors='pt').to('cuda:0')
  generated = model.generate(encoded_input.input_ids, attention_mask = encoded_input.attention_mask, pad_token_id = 26500, do_sample=False, max_length=32, temperature = 1.5, top_k = 1).to('cuda:0')
  results.append(tokenizer.batch_decode(generated, skip_special_tokens=True))

100%|██████████| 2000/2000 [03:20<00:00,  9.96it/s]


# Evaluate model using BLEU score

In [ ]:
# Compare predicted sentences from test data with actual test data. Refer to NLTK documentation for sentence_bleu
import statistics
from nltk.translate.bleu_score import sentence_bleu
bleu_score = 0
for i,j in zip(X_test, results):
  bleu_score += sentence_bleu([j[0].split()], i.split())
print(score/len(X_test))

/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.8/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

0.28443647880268896